## Inputs
---

In [ ]:
# reading data and working with arrays
import h5py, nrrd
import numpy as np

# data paths
data_path = '/groups/scicompsoft/home/fleishmang/rnr-exm-work/data/c_elegan_pair3.h5'
hdf5_file = h5py.File(data_path, 'r')
fix = hdf5_file['/fixed'][...]
mov = hdf5_file['/move'][...]

# assumed spacings
fix_spacing = np.array([0.4, 0.1625, 0.1625])
mov_spacing = np.array([0.4, 0.1625, 0.1625])

# # write some channels
# nrrd.write('./fix.nrrd', fix.transpose(2,1,0), compression_level=2)
# nrrd.write('./mov.nrrd', mov.transpose(2,1,0), compression_level=2)

## Masking
---

### fixed

In [ ]:
# tools for coarse whole brain segmentation
from CircuitSeeker import level_set
from scipy.ndimage import zoom, binary_closing, binary_dilation

# # get small mask
# fix_skip = fix[::4, ::8, ::8]
# skip_spacing = fix_spacing * [4, 8, 8]
# fix_mask_small = level_set.brain_detection(
#     fix_skip, skip_spacing,
#     mask_smoothing=2,
#     iterations=[80,40,10],
#     smooth_sigmas=[12,6,2],
#     lambda2=1.0,
# )

# # enlarge and smooth mask
# fix_mask = zoom(fix_mask_small, np.array(fix.shape) / fix_skip.shape, order=0)
# fix_mask = binary_closing(fix_mask, np.ones((5,5,5))).astype(np.uint8)
# fix_mask = binary_dilation(fix_mask, np.ones((5,5,5))).astype(np.uint8)

# # write result
# nrrd.write('./fix_mask.nrrd', fix_mask.transpose(2,1,0), compression_level=2)

# load precomputed mask
fix_mask, _ = nrrd.read('./fix_mask.nrrd')
fix_mask = fix_mask.transpose(2,1,0)

### moving

In [ ]:
# tools for coarse whole brain segmentation
from CircuitSeeker import level_set
from scipy.ndimage import zoom, binary_closing, binary_dilation

# # get small mask
# mov_skip = mov[::4, ::8, ::8]
# skip_spacing = mov_spacing * [4, 8, 8]
# mov_mask_small = level_set.brain_detection(
#     mov_skip, skip_spacing,
#     mask_smoothing=2,
#     iterations=[80,40,10],
#     smooth_sigmas=[12,6,2],
#     lambda2=1.0,
# )

# # enlarge and smooth mask
# mov_mask = zoom(mov_mask_small, np.array(mov.shape) / mov_skip.shape, order=0)
# mov_mask = binary_closing(mov_mask, np.ones((5,5,5))).astype(np.uint8)
# mov_mask = binary_dilation(mov_mask, np.ones((5,5,5))).astype(np.uint8)

# # save output
# nrrd.write('./mov_mask.nrrd', mov_mask.transpose(2,1,0), compression_level=2)

# load precomputed mask
mov_mask, _ = nrrd.read('./mov_mask.nrrd')
mov_mask = mov_mask.transpose(2,1,0)

## Alignment
---

### global

In [ ]:
%%time

# alignment functions
from bigstream.align import alignment_pipeline
from bigstream.transform import apply_transform

# define alignment steps
common_kwargs = {
    'alignment_spacing':1.0,
    'shrink_factors':(2,),
    'smooth_sigmas':(1.5,),
    'optimizer_args':{
        'learningRate':0.25,
        'minStep':0.,
        'numberOfIterations':3600,
    },
}

affine_kwargs = {
    'initial_condition':'CENTER',
    'fix_mask':fix_mask,
    'mov_mask':mov_mask,
}

steps = [('affine', {**common_kwargs, **affine_kwargs},),]

# align
affine = alignment_pipeline(
    fix, mov,
    fix_spacing, mov_spacing,
    steps,
)

# apply affine only
affine_aligned = apply_transform(
    fix, mov,
    fix_spacing, mov_spacing,
    transform_list=[affine,],
)

# # write results
np.savetxt('affine.mat', affine)
nrrd.write('./affine.nrrd', affine_aligned.transpose(2,1,0), compression_level=2)

# load precomputed results
affine = np.loadtxt('./affine.mat')